In [ ]:
from hypergraph import *

# Graf
Hipergrafy są reprezentowane przez instancje klasy `Graph`. Składa się ona z węzłów (klasa `Node`), które odpowiadają wierzchołkom oraz hiperkrawędziom. Każdy węzeł ma poniższe właściwości:
- `pos` - Pozycja
- `label` - Etykieta
- `attrs` - Słownik zawierający atrybuty
- `is_hyperedge` - `True` jeżeli węzeł jest hiperkrawędzią, `False` w przeciwnym przypadku

W grafie każdy węzeł posiada unikalny identyfikator liczbowy oraz listę węzłów, z którymi jest połączony.

Przy tworzeniu grafu przydatne są poniższe funkcje:

`vertex(pos: tuple[int, int], label: str = "", neighbours: Optional[list[int]] = None, attrs: Optional[dict[str, str]] = None
)`

`hyperedge(pos: tuple[int, int], label: str, neighbours: list[int], attrs: Optional[dict[str, str]] = None)`

Poniżej podany jest przykład prostego grafu.

In [ ]:
starting_graph = Graph({
  0: vertex((0, -1)),
  1: vertex((0,  1)),
  2: hyperedge((-0.1, 0), "A", [0, 1], attrs = { "x": 0, "y": 0 })
})
display(starting_graph)

Podawanie sąsiadów jest wymagane jedynie w przypadku hiperkrawędzi.

# Produkcje
Produkcje są reprezentowane przez instancje klasy `Production`. Zawiera ona grafy lewej oraz prawej strony.

Węzły z równymi identyfikatorami po lewej i po prawej stronie produkcji zachowują atrybuty oraz poprzednio istniejące połączenia z resztą grafu po zastosowaniu reguły.

__Uwaga__, w przeciwieństwie do klasycznej definicji gramatyk hipergrafowych, gdzie transformacja ta dotyczy tylko wierzchołków zewnętrznych w tej bibliotece działa to również dla hiperkrawędzi.

In [ ]:
prod = Production(
    left = Graph({
        0: vertex((0, -1)),
        1: vertex((0,  1)),
        2: hyperedge((-0.1, 0), "A", [0, 1])
    }),
    right = Graph({
        0: vertex((0, -1)),
        1: vertex((0,  1)),
        2: hyperedge((0, 0), "X", [0, 1]),
        3: vertex((2, -1)),
        4: vertex((2,  1)),
        5: hyperedge((1, -1), "B", [0, 3]),
        6: hyperedge((1,  1), "B", [1, 4]),
        7: hyperedge((1.9,  0), "A", [3, 4]),
    }),
    seed_node = 2
)
display(prod)

Produkcję można zastosować przy pomocy metody `apply_once`. Przyjmuje ona graf jako argument i zwraca nowy graf. Jeżeli nie istnieje miejsce, gdzie produkcja może być zastosowana, zostaje zwrócony niezmieniony graf. Oryginalny graf nigdy nie zostaje modyfikowany.

In [ ]:
graph = starting_graph
graph = prod.apply_once(graph)
display(graph)

## Ustalanie pozycji
Pozycje węzłów po zastosowaniu produkcji zostają określone na podstawie pozycji określonych w lewej i prawej stronie produkcji oraz pozycji węzłów usuwanych z grafu. Następuje to według poniższego algorytmu:

1. Znajdź przekształcenie afiniczne (dowolne złożenie obrotu, skalowania i przesunięcia) które mapuje pozycje trzech pierwszych węzłów lewej strony produkcji na pozycje odpowiadających im węzłów w modyfikowanym grafie
2. Zastosuj to przekształcenie dla każdego węzła z prawej strony produkcji

W związku z taką implementacją w każdej produkcji po lewej stronie trzy pierwsze węzły (o indeksach `0, 1, 2`) nie mogą być współliniowe, żeby transformacja wyznaczana w punkcie była jednoznaczna.

## Atrybuty
Produkcje mogą przypisywać atrybuty węzłom z prawej strony. Jest to możliwe poprzez podanie argumentu `attributes` przy tworzeniu produkcji. Wartością tego argumentu powinna być funkcja, która przyjmuje słownik zawierający wszystkie węzły zamieniane w grafie, identyfikowane po indeksach z lewej strony produkcji. Funkcja powinna zwrócić słownik zawierający zestaw atrybutów (jako słownik) dla każdego węzła z prawej strony, który powinien otrzymać atrybuty. Jeżeli węzeł nie występuje w tym słowniku to zachowuje swoje atrybuty.

Poniżej pokazana jest poprzednia produkcja zmodyfikowana tak, żeby ustalała atrybut x i y dla nowej hiperkrawędzi:

In [ ]:
prod = Production(
    left = Graph({
        0: vertex((0, -1)),
        1: vertex((0,  1)),
        2: hyperedge((-0.1, 0), "A", [0, 1])
    }),
    right = Graph({
        0: vertex((0, -1)),
        1: vertex((0,  1)),
        2: hyperedge((0, 0), "X", [0, 1]),
        3: vertex((2, -1)),
        4: vertex((2,  1)),
        5: hyperedge((1, -1), "B", [0, 3]),
        6: hyperedge((1,  1), "B", [1, 4]),
        7: hyperedge((1.9,  0), "A", [3, 4]),
    }),
    attributes = lambda n: {
        7: {
            "x": n[2].attrs["x"] + 1,
            "y": n[2].attrs["y"]
        }
    },
    seed_node = 2
)
display(prod)

In [ ]:
graph = starting_graph
graph = prod.apply_many(graph, max_applications = 2) # max_applications ogranicza ile razy produkcja zostanie zastosowana,
                                                     # bez tego argumentu program wpadnie w nieskończoną pętlę
display(graph)

## Stosowalność produkcji

Można określić predykat ograniczający stosowalność produkcji podając argument `predicate` przy tworzeniu produkcji. Argument ten powinien być funkcją, która przyjmuje taki sam argument, jak `attributes` i zwraca wartość `True`, jeżeli produkcja może być zastosowana, `False` w przeciwnym przypadku. 

Poniżej przedstawiona jest poprzednia produkcja z modyfikacją, która pozwala ją zastosować jedynie jeśli atrybut _x_ jest mniejszy od 3.

In [ ]:
prod = Production(
    left = Graph({
        0: vertex((0, -1)),
        1: vertex((0,  1)),
        2: hyperedge((-0.1, 0), "A", [0, 1])
    }),
    right = Graph({
        0: vertex((0, -1)),
        1: vertex((0,  1)),
        2: hyperedge((0, 0), "X", [0, 1]),
        3: vertex((2, -1)),
        4: vertex((2,  1)),
        5: hyperedge((1, -1), "B", [0, 3]),
        6: hyperedge((1,  1), "B", [1, 4]),
        7: hyperedge((1.9,  0), "A", [3, 4]),
    }),
    attributes = lambda n: {
        7: {
            "x": n[2].attrs["x"] + 1,
            "y": n[2].attrs["y"]
        }
    },
    predicate = lambda n: n[2].attrs["x"] < 3,
    seed_node = 2
)
display(prod)

In [ ]:
graph = starting_graph
graph = prod.apply_many(graph) # max_applications nie jest już potrzebne, ponieważ po zastosowaniu 3 razy 
                               # produkcja przestaje być stosowalna
display(graph)

## Wyświetlanie możliwych miejsc zastosowania
Metody `apply_once` oraz `apply_many` nie specyfikują w którym miejscu produkcja zostanie zastosowana, zależy to od kolejności przeszukiwania węzłów. Przy badaniu gramatyki przydatne może być zobaczenie wszystkich miejsc, gdzie dana produkcja może być zastosowana. Służy do tego metoda `apply_all_possible`, która pozwala przeiterować po wszystkich grafach, które mogą powstać przez zastosowanie produkcji do danego grafu jeden raz:

In [ ]:
prod2 = Production(
  left = Graph({
    0: vertex((-1, 0)),
    1: vertex((1, 0)),
    2: hyperedge((-1, -1), "X", [0]),
    3: hyperedge((0, 0), "B", [0, 1])
  }), 
  right = Graph({
    0: vertex((-1, 0)),
    1: vertex((1, 0)),
    2: hyperedge((-1, -1), "X", [0]),
    3: hyperedge((0, 0), "X", [0, 1]),
    
    4: vertex((-1, 2)),
    5: vertex((1, 2)),
    6: hyperedge((-1, 1), "X", [0, 4]),
    7: hyperedge((1, 1), "X", [1, 5]),
    8: hyperedge((0, 2), "B", [4, 5])
  }),
  seed_node = 3,
)
display(prod2)

In [ ]:
display(graph)

In [ ]:
for g in prod2.apply_all_possible(graph):
    display(g)

Należy zwrócić uwagę na to, że każdy graf, oprócz ostatnich dwóch, jest powtórzony dwa razy. Jest to spowodowane tym, że wierzchołki 0 i 1 mogą być odpowiednio po lewej i prawej, albo przeciwnie. Produkcja nie pozwala jednoznacznie wybrać interpretacji.

# Przykładowa gramatyka

Przykładowa gramatyka generująca szachownicę o zadanych wymiarach

In [ ]:
width = 4
height = 4

In [ ]:
starting_graph = Graph({
    0: vertex((-1, -1)),
    1: vertex((-1,  1)),
    2: vertex(( 1,  1)),
    3: vertex(( 1, -1)),
    4: hyperedge((0, 0), "S", [0, 1, 2, 3], attrs = { "x": 0, "y": 0, "color": 0 }),
    5: hyperedge((-1,  0), "O", [0, 1]),
    6: hyperedge(( 0,  1), "O", [1, 2]),
    7: hyperedge(( 1,  0), "V", [2, 3]),
    8: hyperedge(( 0, -1), "H", [3, 0])
})
display(starting_graph)

In [ ]:
prod1 = Production(
  left = Graph({
    0: vertex(( 0,  1)),
    1: vertex(( 0, -1)),
    2: hyperedge((-1, 0), "S", [0, 1]),
    3: hyperedge(( 0, 0), "V", [0, 1]),
    4: hyperedge((-1, 1), "O", [0]),
  }), 
  right = Graph({
    0: vertex(( 0,  1)),
    1: vertex(( 0, -1)),
    2: hyperedge((-1, 0), "S", [0, 1]),
    3: hyperedge(( 0, 0), "E", [0, 1]),
    4: hyperedge((-1, 1), "O", [0]),
    5: vertex(( 2,  1)),
    6: vertex(( 2, -1)),
    7: hyperedge((1,  1), "O", [0, 5]),
    8: hyperedge((1, -1), "H", [1, 6]),
    9: hyperedge((2,  0), "V", [5, 6]),
    10: hyperedge((1,  0), "S", [0, 1, 5, 6]),
  }),
  attributes = lambda n: {
    10: { 
      "x": n[2].attrs["x"] + 1,
      "y": n[2].attrs["y"],
      "color": 1 - n[2].attrs["color"]
    }
  },
  predicate = lambda n: n[2].attrs["x"] + 1 < width,
  seed_node = 3,
)
display(prod1)

In [ ]:
graph = starting_graph
graph = prod1.apply_many(graph)
display(graph)

In [ ]:
prod2 = Production(
  left = Graph({
    0: vertex(( 1,  0)),
    1: vertex((-1, 0)),
    2: hyperedge((0, 1), "S", [0, 1]),
    3: hyperedge((0, 0), "H", [0, 1]),
    4: hyperedge((-1, 1), "O", [1])
  }), 
  right = Graph({
    0: vertex(( 1, 0)),
    1: vertex((-1, 0)),
    2: hyperedge((0, 1), "S", [0, 1]),
    3: hyperedge((0, 0), "E", [0, 1]),
    4: hyperedge((-1, 1), "O", [1]),
    5: vertex(( 1, -2)),
    6: vertex((-1, -2)),
    7: hyperedge(( 1, -1), "V", [0, 5]),
    8: hyperedge((-1, -1), "O", [1, 6]),
    9: hyperedge(( 0, -2), "H", [5, 6]),
    10: hyperedge(( 0, -1), "S", [0, 1, 5, 6])
  }),
  attributes = lambda n: {
    10: { 
      "x": n[2].attrs["x"],
      "y": n[2].attrs["y"] + 1,
      "color": 1 - n[2].attrs["color"]
    }
  },
  predicate = lambda n: n[2].attrs["y"] + 1 < height and n[2].attrs["x"] == 0,
  seed_node = 3,
)
display(prod2)

In [ ]:
graph = prod2.apply_many(graph)
display(graph)

In [ ]:
prod3 = Production(
  left = Graph({
    0: vertex(( 1,  0)),
    1: vertex((-1, 0)),
    2: vertex((-1, -2)),    
    3: hyperedge((0, 1), "S", [0, 1]),    
    4: hyperedge((-2, -1), "S", [1, 2]),
    5: hyperedge((0, 0), "H", [0, 1]),
    6: hyperedge((-1, -1), "V", [1, 2])
  }), 
  right = Graph({
    0: vertex(( 1,  0)),
    1: vertex((-1, 0)),
    2: vertex((-1, -2)),    
    3: hyperedge((0, 1), "S", [0, 1]),    
    4: hyperedge((-2, -1), "S", [1, 2]),
    5: hyperedge((0, 0), "E", [0, 1]),
    6: hyperedge((-1, -1), "E", [1, 2]),
    
    7: vertex((1, -2)),
    8: hyperedge((0, -2), "H", [2, 7]),
    9: hyperedge((1, -1), "V", [0, 7]),
    10: hyperedge((0, -1), "S", [0, 1, 2, 7])
  }),
  attributes = lambda n: {
    10: { 
      "x": n[3].attrs["x"],
      "y": n[4].attrs["y"],
      "color": 1 - n[3].attrs["color"]
    }
  },
  seed_node = 5,
)
display(prod3)

In [ ]:
graph = prod3.apply_many(graph)
display(graph)

Można zapisać każdy krok wywodu przy pomocy metody `save_svg`

In [ ]:
width = 3
height = 3

step = 0

def output_graph(graph):
  global step
  graph.save_svg(f"steps/{step:03d}.svg")
  step += 1

graph = starting_graph
output_graph(graph)
  
for production in [prod1, prod2, prod3]:
  while True:
    g = production.apply_once(graph)
    if g is None:
      break
    graph = g
    output_graph(graph)